<a href="https://colab.research.google.com/github/fortune901/nubase-project/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!git clone https://github.com/fortune901/nubase-project.git # cloner mon depot github
#%cd nubase-project # va dans le dossier du depot
#!ls #liste les fichiers pour verifier
from google.colab import files
uploaded = files.upload()

Saving AME2020.dat to AME2020.dat


In [2]:

#importation des bibliothèques nécessaires

import pandas as pd # pour lire et manipuler les données tabulaires (fichier.csv)
from sklearn.model_selection import train_test_split # pour diviser les données en ensemble d'entrainement et de test
from sklearn.preprocessing import StandardScaler # pour normaliser les données
from tensorflow.keras.models import Sequential # ici c'est pour construire un modèle de reseau de neurone
from tensorflow.keras.layers import Dense # ici c'est pour pouvoir ajouter des couches de neurones cachees
#import re
# Fonction de conversion d'une demi-vie texte en secondes
#def convertir_demi_vie_en_seconndes(texte):
  #"""
  #Convertit une demi-vie sous forme de texte (ex. '2.5 ms) en secondes.
  #"""
  #if not texte or texte.lower() =='stbl':
   # return None
    #texte = texte.strip().replace('-', '-').replace('µ', 'u') # Nettoyage
    #match = re.match(r'^([0-9.eE+\-]+)\s*([a-zA-Z]+)$', texte)
    #if not match:
     # return None
    #valeur, unite = match.groups()
    #try:
      #valeur = float(valeur)
    #except ValueError:
      #return None
      #facteurs ={'s': 1,
                 #'ms': 1e-3,
                 #'us': 1e-6,
                 #'ns': 1e-9,
                 #'ps': 1e-12,
                 #'m': 60,
                 #'h': 3600,
                 #'d': 86400,
                 #'y': 365.25* 86400,
                 #'ky': 1e3* 365.25* 86400,
                 #'My': 1e6* 365.25* 86400,
                 #'Gy': 1e9* 365.25* 86400,
                 #}
    #unite = unite.lower()
    #if unite not in facteurs:
      #return None
      #return valeur * facteurs[unite]
import re

def convertir_demi_vie_en_secondes(demi_vie_texte):
    if not demi_vie_texte or demi_vie_texte.strip() == '':
        return None

    # Dictionnaire des unités vers secondes
    conversion = {
        's': 1,
        'ms': 1e-3,
        'us': 1e-6,
        'ns': 1e-9,
        'min': 60,
        'h': 3600,
        'H': 3600,
        'd': 86400,
        'y': 31536000,
        'a': 31536000,  # parfois y = a = année
    }

    # Extraction avec expression régulière
    match = re.match(r'([\d\.]+)\s*([a-zA-Zµ]+)', demi_vie_texte.strip())
    if match:
        valeur = float(match.group(1))
        unite = match.group(2).lower()  # on passe l’unité en minuscule

        for cle in conversion:
            if cle.lower() == unite:
                return valeur * conversion[cle]

    return None  # si non reconnu
#Extraction des donnees
def extraire_donnees_nubase(fichier):
  """
  Extrait les donnees nucleaires depuis le fichier nubase_4.mas20.
  """
  donnees = []
  with open(fichier, 'r', encoding='utf-8') as f:
    for ligne in f:
      if ligne.strip() == '' or ligne.startswith('#'): # cette ligne de code permet de sauter les lignes vides ou de commmentaire
        continue
      try:
        #Lectures des champs depuis les positions fixes (selon la structure de nubase2020)
        # attention : les indices peuvent varier selon les versions du fichier
        Z = ligne[0:3].strip() # Numero atomique (proton)
        N = ligne[4:8].strip() # Nombre de neutrons
        A = ligne[9:13].strip() # Nombre de masse
        element = ligne[14:17].strip() # Symbole de l'element
        Q= ligne[23:37].strip() # état fonfamentale (g) ou isomere (m)
        #demi_vie_texte = ligne[59:80].strip() # Demi-vie (au format texte, souvent log ou unité ex: '1.23 s')
        #if demi_vie_texte.lower() =='stbl': #ici on saute les noyaux stable
            #continue
          #demi_vie_s = None # Initialisation par defaut
        #try:
          #demi_vie_s = convertir_demi_vie_en_secondes(demi_vie_texte) # convertir en secondes
        #except:
            #demi_vie_s = None
            #pass  # ignore l'erreur si ca ne convertit pas
        #deformation= ligne[90:100].strip() # Deformation nucleaire (si disponible)
        #shell_effect = ligne[100:110].strip() # Effet de coquille (ex: energie de liaison)
        #spin_orbite = ligne[110:120].strip() # Info sur le spin ou couplage (selon le format exact)
        # Ajoutons les données sous formes de dictionnaire

        donnees.append({'Z': Z,
                       'N': N,
                       'A': A,
                       'Element': element,
                       #'Etat': etat,
                       'Q(KeV)': Q
        })
                        #'Demi-vie (s)': demi_vie_s, #ajout de la demi-vie en secondes
                       #'Deformation': deformation,
                       #'Shell_effect': shell_effect,
                       #'Spin_orbite': spin_orbite})
      except IndexError: # si une ligne est trop courte on l'ignore
        continue
  return donnees
# Utilisation du script
#fichier = 'nubase_4.mas20'
#donnees = extraire_donnees_nubase(fichier)
#affichage des 5 premiers lignes extraites
#for ligne in donnees[:5]:
  #print(ligne)
# Utilisation du script
fichier = 'AME2020.dat'#'nubase_4.mas20'
donnees = extraire_donnees_nubase(fichier)
#affichage de toutes les lignes extraites
for ligne in donnees:
  print(ligne)


{'Z': 'N', 'N': 'Z', 'A': 'A', 'Element': 'El', 'Q(KeV)': 'ass'}
{'Z': '1', 'N': '0', 'A': '1', 'Element': 'n', 'Q(KeV)': '008664.91590'}
{'Z': '0', 'N': '1', 'A': '1', 'Element': 'H', 'Q(KeV)': '007825.031898'}
{'Z': '1', 'N': '1', 'A': '2', 'Element': 'H', 'Q(KeV)': '014101.777844'}
{'Z': '2', 'N': '1', 'A': '3', 'Element': 'H', 'Q(KeV)': '016049.28132'}
{'Z': '1', 'N': '2', 'A': '3', 'Element': 'He', 'Q(KeV)': '016029.32197'}
{'Z': '0', 'N': '3', 'A': '3', 'Element': 'Li', 'Q(KeV)': '030775#'}
{'Z': '3', 'N': '1', 'A': '4', 'Element': 'H', 'Q(KeV)': '026431.867'}
{'Z': '2', 'N': '2', 'A': '4', 'Element': 'He', 'Q(KeV)': '002603.25413'}
{'Z': '1', 'N': '3', 'A': '4', 'Element': 'Li', 'Q(KeV)': '027185.561'}
{'Z': '4', 'N': '1', 'A': '5', 'Element': 'H', 'Q(KeV)': '035311.492'}
{'Z': '3', 'N': '2', 'A': '5', 'Element': 'He', 'Q(KeV)': '012057.224'}
{'Z': '2', 'N': '3', 'A': '5', 'Element': 'Li', 'Q(KeV)': '012537.800'}
{'Z': '1', 'N': '4', 'A': '5', 'Element': 'Be', 'Q(KeV)': '039870#

In [3]:
import csv
# Enregistrement dans un fichier CSV
with open('donnees_nubase.csv', 'w', newline='', encoding='utf-8') as csvfile:
  champs = ['Z', 'N','A','Element',  'Q(KeV)']
  writer = csv.DictWriter(csvfile, fieldnames=champs)
  writer.writeheader()
  writer.writerows(donnees)

# cONVERTIR LA LISTE DE DICTIONNAIRES EN DATAFRAME
df = pd.DataFrame(donnees)
df
df.to_csv("donnees_nubase.csv", index=False)
df[['N','Z', 'Q(KeV)']]
from google.colab import files
files.download("donnees_nubase.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
df = pd.read_csv('donnees_nubase.csv')
df.head()
#supprimer les colonnes qui ne sont pas utile (optionnel)
#df.drop(columns=['Colonneinutile1', 'colonneinutile2'], inplac=True)
print(df.dtypes) #Verfier les types de données

Z          object
N          object
A          object
Element    object
Q(KeV)     object
dtype: object


In [5]:
# Supprimer les espaces en trop
df.columns = df.columns.str.strip()
print(df.isnull().sum()) # regarde les valeurs manquantes

Z          0
N          0
A          0
Element    0
Q(KeV)     0
dtype: int64


In [6]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd # Ensure pandas is imported

# Cherchons les colonnes numeriques à normaliser
scaler = MinMaxScaler()
cols_a_normaliser = ['Z', 'N', 'A', 'Q(KeV)']

# Convertir les colonnes en type numérique, en gérant les erreurs
for col in cols_a_normaliser:
    # errors='coerce' remplacera les valeurs non convertibles par NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Gérer les valeurs manquantes (NaN) créées par la conversion
# Option 1: Remplacer les NaNs par une valeur (ex: la moyenne, la médiane, ou 0)
# df.fillna(df.mean(), inplace=True) # Remplace par la moyenne de chaque colonne
# df.fillna(0, inplace=True)       # Remplace par 0

# Option 2: Supprimer les lignes contenant des NaNs dans ces colonnes
df.dropna(subset=cols_a_normaliser, inplace=True) # Supprime les lignes si NaN dans l'une des colonnes spécifiées

# Maintenant, appliquer le scaler sur les colonnes nettoyées
df[cols_a_normaliser] = scaler.fit_transform(df[cols_a_normaliser])

print(df.head())
print(df.describe())

         Z         N         A Element    Q(KeV)
1  0.00625  0.000000  0.000000       n  0.008665
2  0.00000  0.009091  0.000000       H  0.007825
3  0.00625  0.009091  0.003717       H  0.014102
4  0.01250  0.009091  0.007435       H  0.016050
5  0.00625  0.018182  0.007435      He  0.016030
                 Z            N            A       Q(KeV)
count  2550.000000  2550.000000  2550.000000  2550.000000
mean      0.482919     0.502809     0.489130     0.759872
std       0.248338     0.236930     0.243151     0.362975
min       0.000000     0.000000     0.000000     0.000000
25%       0.287500     0.318182     0.301115     0.906501
50%       0.487500     0.509091     0.498141     0.927268
75%       0.668750     0.709091     0.680297     0.952533
max       1.000000     1.000000     1.000000     1.000000


In [7]:
from sklearn.model_selection import train_test_split

X = df[['Z', 'N', 'A']]  # Ajoute d'autres colonnes si besoin
y = df['Q(KeV)']

# Exemple de séparation
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

# choix du modèle de ML
#from sklearn.ensemble import RandomForestRegressor

#model = RandomForestRegressor(n_estimators=100, random_state=42)
#model.fit(X_train, y_train)
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', max_iter=1000)
model.fit(X_train, y_train)

# validation croisée
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
# evaluation du modèle
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R²: {r2}")

#prediction de Q
Q_pred = model.predict(X_test)
print(Q_pred)

RMSE: 0.17460509583088504
R²: 0.736161531403202
[ 0.9345434   0.41008948  0.646314    0.92675863  0.18790738  0.94240163
  0.93918078  0.37558279  0.93103826  0.92908393  0.93356533  0.12585344
  0.92574878  0.51447502  0.8167417   0.16087317  0.78718404  0.06572132
  0.94213011  0.92850043 -0.07163629  0.93544919  0.84990259  0.93416311
  0.9248239   0.54839318  0.94363877  0.94840459  0.20156398  0.95020625
  0.93885918  0.93993241  0.94020718  0.94623745  0.94492567  0.05461882
  0.94868685  0.72254665  0.59090866  0.27014567  0.92807929  0.94675221
  0.93635999  0.92570271  0.93078894  0.94186199  0.931396    0.94480489
  0.65035815  0.59305904  0.93094311  0.78900801  0.93964509  0.6908875
  0.93789438  0.92913153  0.92662733  0.82735086  0.93135986  0.93463855
  0.92472676  0.92824861  0.92866869  0.94593031  0.94742483  0.86671588
  0.94600495  0.92906598  0.93201634  0.92886894  0.05045682  0.93779922
  0.93441211  0.95172565  0.27660974  0.94139699  0.92453595  0.94019499
  0.

In [8]:
#evaluation du modèle de prediction
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

rmse = np.sqrt(mean_squared_error(y_test, Q_pred))
r2 = r2_score(y_test, Q_pred)

print(f"RMSE: {rmse}")
print(f"R²: {r2}")

RMSE: 0.17460509583088504
R²: 0.736161531403202


In [12]:
#prediction des demi-vies
def predict_half_life(Q_alpha, Z, a, b, c, d):
    # Ensure Z and Q_alpha are numpy arrays for element-wise operations
    Z = np.asarray(Z)
    Q_alpha = np.asarray(Q_alpha)

    # Handle potential division by zero or negative Q_alpha values
    # Replacing non-positive Q_alpha with a small positive number to avoid errors
    Q_alpha[Q_alpha <= 0] = 1e-9 # Replace with a small positive value

    log_T = (a * Z + b) / np.sqrt(Q_alpha) + (c * Z + d)
    T = 10**log_T  # Convertir log(T) en T
    return T

# Exemple d'utilisation
# Use the 'Z' column from the X_test DataFrame
predicted_half_lives = predict_half_life(Q_pred, X_test['Z'], a=-0.5, b=20, c=0.1, d=-5)
print(predicted_half_lives)

[2.79289181e+015 4.90876932e+025 6.84190497e+019 3.63359798e+015
 1.64497612e+040 2.95002097e+015 2.39916250e+015 3.96936676e+027
 3.16284519e+015 3.79442553e+015 2.93202997e+015 1.63799418e+050
 5.35666241e+015 2.72721766e+022 1.17713606e+017 6.57428192e+043
 1.56425473e+017 2.11989209e+071 3.36709568e+015 3.58246112e+015
             inf 2.73011509e+015 4.35506148e+016 2.53078464e+015
 4.17970875e+015 3.61501101e+021 2.84335665e+015 2.58697774e+015
 4.78133868e+038 2.57944185e+015 2.40402598e+015 2.29901072e+015
 3.49157431e+015 2.88010532e+015 2.83706662e+015 5.85264149e+078
 2.63083649e+015 2.97665060e+018 3.96315859e+020 5.44718044e+032
 3.86937314e+015 2.87756796e+015 2.42435620e+015 5.33073052e+015
 3.08848510e+015 2.90204541e+015 3.11557343e+015 3.14923192e+015
 2.55189709e+019 3.70590256e+020 3.15143947e+015 1.54517460e+017
 3.12308574e+015 4.98979584e+018 2.41869851e+015 3.29935626e+015
 3.66784660e+015 4.53227987e+016 3.15617271e+015 2.80300380e+015
 3.99235680e+015 3.477375

/tmp/ipython-input-12-1677709797.py:12: RuntimeWarning: overflow encountered in power
  T = 10**log_T  # Convertir log(T) en T
